Scrapes the novelupdates webpage, http://www.novelupdates.com/
for novel information and cleans it.
Finally saves the dataset as a csv file.

Didn't find any easy way to get all the novel ids, hence, the novels listing is first used to get all ids.

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import re

novel_list_page = "http://www.novelupdates.com/novelslisting/?st=1&pg="
novel_page = "http://www.novelupdates.com/?p="

In [2]:
# Get the maximum number of novel pages from the listings

def get_novel_list_max_pages(page):
    soup = BeautifulSoup(page.content, 'html.parser')
    dig_pag = soup.find('div', attrs={'class':'digg_pagination'})
    page_links = dig_pag.find_all('a')
    last_page_link = str(page_links[2]) # The last page is the 3rd
    num = re.search('pg=\d+', last_page_link).group()[3:]
    return int(num)

page = requests.get(novel_list_page + '1')
novels_max_pages = get_novel_list_max_pages(page)

In [4]:
novels_max_pages = 1

# Get all novel ids from the novels listing
def get_novel_ids(page):
    soup = BeautifulSoup(page.content, 'html.parser')
    table = soup.find('table', attrs={'id':'myTable'})
    table_body = table.find('tbody')
    rows = table_body.find_all('tr')
    
    novel_ids = []
    for row in rows:
        col = row.find_all('td')[-1]
        novel_id = col.a['id'][3:]
        novel_ids.append(novel_id)
    return novel_ids

all_novel_ids = []
for i in range(1,novels_max_pages+1):
    page = requests.get(novel_list_page + str(i))
    novel_ids = get_novel_ids(page)
    all_novel_ids.extend(novel_ids)
    time.sleep(1)

df = pd.DataFrame(all_novel_ids, columns=['id'])

In [19]:
def parse_novel_page(id):
#    page = requests.get(novel_page + str(id))
    page = requests.get(id) # TEMP
    soup = BeautifulSoup(page.content, 'html.parser')
    wrapper = soup.find('div', attrs={'class': 'wpb_wrapper'})
    
    #tags =
    
    genres = [genre.text
              for genre in wrapper
                .find('div', attrs={'id': 'seriesgenre'})
                .find_all('a', attrs={'class': 'genre'})]

    
    #rating =
    #rating_votes =
    #langauge =
    #author = 
    #start_year = 
    #volumes_total =
    #chapters_total =
    #chapters_translated =
    #complete
    #complete_translated =
    #licensed = 
    #original_publisher =
    #english_publisher =
    #release_freq = 
    #activity_week_rank =
    #activity_month_rank =
    #activity_all_time_rank =
    #on_reading_lists =
    #reading_list_month_rank =
    #reading_list_all_time_rank = 
    #assoc_names = 
    #related_series_ids =
    #recommended_series_ids =
    
    
    # return these as a map (name -> result)
    # add these as new columns to the df
    time.sleep(1)
    return 3

#https://stackoverflow.com/questions/16236684/apply-pandas-function-to-column-to-create-multiple-new-columns
parse_novel_page('http://www.novelupdates.com/series/i-shall-seal-the-heavens/')

# div id="seriesgenre" / class="genre"
# <a class="genre" gid="8" href="http://www.novelupdates.com/genre/action/" title="A work typically depicting fighting, violence, chaos, and fast paced motion.">Action</a>

# h5 class=seriesother / class="uvotes"
# <span class="uvotes">(4.4 / 5.0, 2481 votes)</span>
    
#df['test'] = df.apply(lambda x: parse_novel_page(x['Id']), axis=1)
#df.head

['Action', 'Adventure', 'Comedy', 'Drama', 'Fantasy', 'Martial Arts', 'Romance', 'Xianxia']


3